In [26]:
import json
import math

# Load JSON data
with open("output/gaze_scoring_results_new.json") as f:
    data = json.load(f)

# Basic utilities
def euclidean(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def final_point(seq):
    for pt in reversed(seq):
        if pt is not None:
            return pt
    return None

def denormalize_point(point, input_size=(100, 100), target_size=(512, 320)):
    x, y = point
    sx, sy = input_size
    tx, ty = target_size
    return [x / sx * tx, y / sy * ty]

def in_bbox(pt, bbox):
    x, y = pt
    x0, y0, w, h = bbox
    return x0 <= x <= x0 + w and y0 <= y <= y0 + h

# Shared metadata
TARGET_SIZE = (512, 320)


In [27]:
metric = "final_dist_to_center"
results = {"top1": 0, "top3": 0}

for entry in data:
    x0, y0, w, h = entry["bbox"]
    center = (x0 + w / 2, y0 + h / 2)
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pt = final_point(seq)
        if pt is not None:
            pt = denormalize_point(pt, target_size=TARGET_SIZE)
            dist = euclidean(pt, center)
        else:
            dist = float("inf")
        scores.append((dist, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


final_dist_to_center {'top1': 32, 'top3': 69}


In [28]:
metric = "avg_dist_to_center"
results = {"top1": 0, "top3": 0}

for entry in data:
    x0, y0, w, h = entry["bbox"]
    center = (x0 + w / 2, y0 + h / 2)
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]
        if pts:
            dist = sum(euclidean(p, center) for p in pts) / len(pts)
        else:
            dist = float("inf")
        scores.append((dist, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


avg_dist_to_center {'top1': 36, 'top3': 64}


In [29]:
metric = "min_dist_to_center"
results = {"top1": 0, "top3": 0}

for entry in data:
    x0, y0, w, h = entry["bbox"]
    center = (x0 + w / 2, y0 + h / 2)
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]
        if pts:
            dist = min(euclidean(p, center) for p in pts)
        else:
            dist = float("inf")
        scores.append((dist, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


min_dist_to_center {'top1': 27, 'top3': 52}


In [30]:
metric = "path_length"
results = {"top1": 0, "top3": 0}

for entry in data:
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]
        if len(pts) > 1:
            dist = sum(euclidean(pts[i], pts[i+1]) for i in range(len(pts) - 1))
        else:
            dist = float("inf")
        scores.append((dist, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


path_length {'top1': 17, 'top3': 34}


In [31]:
metric = "pct_in_bbox"
results = {"top1": 0, "top3": 0}

for entry in data:
    bbox = entry["bbox"]
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]
        if pts:
            pct = sum(in_bbox(p, bbox) for p in pts) / len(pts)
            score = -pct  # invert to make lower better
        else:
            score = float("inf")
        scores.append((score, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


pct_in_bbox {'top1': 17, 'top3': 42}


In [32]:
metric = "norm_path_length"
results = {"top1": 0, "top3": 0}

for entry in data:
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]
        if len(pts) >= 2:
            total = sum(euclidean(pts[i], pts[i+1]) for i in range(len(pts) - 1))
            net = euclidean(pts[0], pts[-1])
            score = total / net if net > 0 else float("inf")
        else:
            score = float("inf")
        scores.append((score, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


norm_path_length {'top1': 16, 'top3': 50}


In [33]:
# Weights can be tuned later
weights = {
    "final_dist": 0.4,
    "avg_dist": 0.4,
    "pct_in_bbox": 0.2
}

metric = "weighted_composite"
results = {"top1": 0, "top3": 0}

for entry in data:
    x0, y0, w, h = entry["bbox"]
    center = (x0 + w / 2, y0 + h / 2)
    scores = []

    for cand, seq in zip(entry["candidates"], entry["gaze_sequences"]):
        pts = [denormalize_point(p, target_size=TARGET_SIZE) for p in seq if p]

        if not pts:
            final_d = avg_d = float("inf")
            pct_in = 0
        else:
            final_d = euclidean(pts[-1], center)
            avg_d = sum(euclidean(p, center) for p in pts) / len(pts)
            pct_in = sum(in_bbox(p, entry["bbox"]) for p in pts) / len(pts)

        score = (
            weights["final_dist"] * final_d +
            weights["avg_dist"] * avg_d -
            weights["pct_in_bbox"] * pct_in  # subtract: higher pct is better
        )
        scores.append((score, cand["type"]))

    ranked = sorted(scores)
    if ranked[0][1] == "gold": results["top1"] += 1
    if any(r[1] == "gold" for r in ranked[:3]): results["top3"] += 1

print(metric, results)


weighted_composite {'top1': 37, 'top3': 68}
